In [ ]:
import pandas as pd
import pickle
import requests as rq
import json
import math

In [ ]:
df = pd.read_pickle("data.pkl")

In [ ]:
with open('isocodes.pkl', 'rb') as handle:
  isocodes = pickle.load(handle)
isocodes

In [ ]:
with open('yidaiyilu_api_data.json', 'r') as handle:
  data = json.load(handle)

In [ ]:
yidaiyilu_data = {}

In [ ]:
# Request part
# Need to add a proxy to get the request - template of request
'''
apiURL = "https://eng.yidaiyilu.gov.cn/wcm.files/data/data-query-20170525.effix"
headers = {
    'Accept': "application/json, text/javascript, */*; q=0.01",
    'Referer': "https://eng.yidaiyilu.gov.cn/dsjym.htm",
    'X-Requested-With': "XMLHttpRequest",
    'Sec-Fetch-Mode': "cors",
    'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36",
    'X-Request-Id': "1150;r=19560479",
    'cache-control': "no-cache",
    'Cookie' : "__jsluid_s=a0854fc5fcc55ebf2f523fc55744cae0; path=/; domain=.eng.yidaiyilu.gov.cn; Secure; HttpOnly; Expires=Tue, 13 Oct 2020 02:15:44 GMT;"
}

resp = rq.get(apiURL, headers)
        
# Error handler
if(resp.status_code != 200) :
    print("STATUS_CODE ERROR", resp.status_code, resp.text)
else:
    # Response handler
    data = resp.json()
    data["country_macro"]
'''

In [ ]:
# 2015 year
year = "65d2ea03425887a717c435081cfc5dbb" 

# useful indicators for scrapping
indicators = {
    "9b688b5fbb441714ba73f4379e7b135d": "BRI GDP",
    "5c4591fe4746333f6e7b66b13ad980a2": "BRI GDP growth rate",
    "6d602224213550f3ee07477f1654f326": "BRI GDP p.c.",
    "07f451a8c9b2006fb937d1eb268ac60b": "BRI Y.o.Y. GDP growth rate p.c.",
    "c1329b4c01d4c08a68b8a280fb5a90ec": "BRI Agriculture value added per sector ",
    "f00273fa66d45147b2dd4dadd6275d32": "BRI Industry value added per sector ",
    "41fedd97a26d4782e531b5b4c67622b7": "BRI Services value added per sector ",
    "7b8d0a597b57bab793f428193eb899fe": "BRI Internet penetration rate (%)",
    "2a048dbe6ff2ee51d07a6314752c87d7": "BRI Population density (/km2)",
    "85bf6dab128029be54a3b1e643f0aa71": "BRI Percentage of Urban Population in Total Population"
}

for key_country in data['country_macro']['term']['area']:
    full_country_name = data['country_macro']['term']['area'][key_country]['full']
    short_country_name = data['country_macro']['term']['area'][key_country]['short']

    # Check if we care about this country
    if full_country_name not in isocodes and short_country_name not in isocodes:
        continue
    else:
        yidaiyilu_data[full_country_name] = {}
    
    for key_indic in data['country_macro']['data'][year][key_country]:
        # Check if we care about this indicator
        if key_indic not in indicators:
            continue
        value = data['country_macro']['data'][year][key_country][key_indic]
        if value == '':
            value = 0
        yidaiyilu_data[full_country_name].update({indicators[key_indic] : value})

In [ ]:
# We check how many countries data we have
len(yidaiyilu_data)

In [ ]:
yidaiyilu_data

In [272]:
# Add data to df
for country in yidaiyilu_data.keys():
    global_rate = 0
    for indic in yidaiyilu_data[country]:
        if pd.isnull(df[indic].iloc[df['Country (according to CN Gov):'][df['Country (according to CN Gov):'] == country].index[0]-1]):
            value = yidaiyilu_data[country][indic]
            df[indic].iloc[df['Country (according to CN Gov):'][df['Country (according to CN Gov):'] == country].index[0]-1] = value
            if "sector" in indicators[indic] :
                global_rate += value
        if global_rate > 0 :
            df["Cumulative value added (numbers should reflect 100%)"].iloc[df['Country (according to CN Gov):'][df['Country (according to CN Gov):'] == country].index[0]-1] = value

In [273]:
# Get the data from HF source
hf_data_source = pd.read_excel('index2019_data.xls', index_col=0)

In [274]:
# Object with final data
hf_data_keep = {}

In [275]:
# Check if we have the right countries
for my_country in isocodes:
    for index, country in hf_data_source['Country Name'].items():
        if country == my_country or country == isocodes[my_country]['other']:
            hf_data_keep[my_country] = {"index" : index}

In [276]:
for country in hf_data_keep:
    rank = hf_data_source.loc[hf_data_keep[country]['index'],'World Rank']
    scale = hf_data_source.loc[hf_data_keep[country]['index'],'2019 Score']
           
    hf_data_keep[country].update({"rank" : rank})
    hf_data_keep[country].update({"scale" : scale})

In [277]:
hf_data_keep

{'Afghanistan': {'index': 1, 'rank': 152.0, 'scale': 51.5},
 'Albania': {'index': 2, 'rank': 52.0, 'scale': 66.5},
 'Armenia': {'index': 6, 'rank': 47.0, 'scale': 67.7},
 'Azerbaijan': {'index': 9, 'rank': 60.0, 'scale': 65.4},
 'Bahrain': {'index': 11, 'rank': 54.0, 'scale': 66.4},
 'Bangladesh': {'index': 12, 'rank': 121.0, 'scale': 55.6},
 'Belarus': {'index': 14, 'rank': 104.0, 'scale': 57.9},
 'Bhutan': {'index': 18, 'rank': 74.0, 'scale': 62.9},
 'Bosnia and Herzegovina': {'index': 20, 'rank': 83.0, 'scale': 61.9},
 'Bulgaria': {'index': 23, 'rank': 37.0, 'scale': 69.0},
 'Cambodia': {'index': 27, 'rank': 105.0, 'scale': 57.8},
 'China': {'index': 34, 'rank': 100.0, 'scale': 58.4},
 'Croatia': {'index': 41, 'rank': 86.0, 'scale': 61.4},
 'Czech Republic': {'index': 44, 'rank': 23.0, 'scale': 73.7},
 'Egypt': {'index': 50, 'rank': 144.0, 'scale': 52.5},
 'Estonia': {'index': 54, 'rank': 15.0, 'scale': 76.6},
 'France (Not BRI)': {'index': 58, 'rank': 71.0, 'scale': 63.8},
 'Georgi

In [278]:
# Add data to df
for country in hf_data_keep.keys():
    if pd.isnull( df['HF Index of Economic Freedom (Scale: 1-100)'].iloc[list(isocodes).index(country)]):
        df['HF Index of Economic Freedom (Scale: 1-100)'].iloc[list(isocodes).index(country)] = hf_data_keep[country]['scale']
    if pd.isnull( df['Heritage Foundation Index of Economic Freedom (Global Ranking)'].iloc[list(isocodes).index(country)]):
        df['Heritage Foundation Index of Economic Freedom (Global Ranking)'].iloc[list(isocodes).index(country)] = hf_data_keep[country]['rank']

In [279]:
# Get the data from BTI source
bti_data_source = pd.read_excel('BTI_2018_Scores.xlsx')

In [280]:
# Object with final data
bti_data_keep = {}

In [281]:
# Check if we have the right countries
for my_country in isocodes:
    for index, country in bti_data_source['Regions:\n1 | East-Central and Southeast Europe\n2 | Latin America and the Caribbean\n3 | West and Central Africa\n4 | Middle East and North Africa\n5 | South and East Africa\n6 | Post-Soviet Eurasia\n7 | Asia and Oceania'].items():
        if country == my_country or country == isocodes[my_country]['other']:
            bti_data_keep[my_country] = {"index" : index}

In [282]:
for country in bti_data_keep:
    rank = bti_data_source.loc[bti_data_keep[country]['index'],'  Ranking Status Index']
    scale = bti_data_source.loc[bti_data_keep[country]['index'],'  S | Status Index']
                               
    bti_data_keep[country].update({"rank" : rank})
    bti_data_keep[country].update({"scale" : scale})

In [283]:
bti_data_keep

{'Afghanistan': {'index': 0, 'rank': 121, 'scale': 2.954761904761905},
 'Albania': {'index': 1, 'rank': 30, 'scale': 6.828571428571428},
 'Armenia': {'index': 5, 'rank': 61, 'scale': 5.576190476190476},
 'Azerbaijan': {'index': 6, 'rank': 100, 'scale': 4.127380952380952},
 'Bahrain': {'index': 7, 'rank': 89, 'scale': 4.616666666666667},
 'Bangladesh': {'index': 8, 'rank': 79, 'scale': 5.1119047619047615},
 'Belarus': {'index': 9, 'rank': 86, 'scale': 4.720238095238095},
 'Bhutan': {'index': 11, 'rank': 45, 'scale': 6.2476190476190485},
 'Bosnia and Herzegovina': {'index': 13,
  'rank': 43,
  'scale': 6.282142857142857},
 'Bulgaria': {'index': 16, 'rank': 17, 'scale': 7.8},
 'Cambodia': {'index': 19, 'rank': 103, 'scale': 3.9976190476190476},
 'China': {'index': 24, 'rank': 81, 'scale': 5.016666666666667},
 'Croatia': {'index': 30, 'rank': 14, 'scale': 8.067857142857143},
 'Czech Republic': {'index': 32, 'rank': 1, 'scale': 9.521428571428572},
 'Egypt': {'index': 35, 'rank': 96, 'scale'

In [284]:
# Add data to df
for country in bti_data_keep.keys():
    if pd.isnull( df['Bertelsmann Transformation index (BTI)'].iloc[list(isocodes).index(country)]):
        df['Bertelsmann Transformation index (BTI)'].iloc[list(isocodes).index(country)] = bti_data_keep[country]['scale']
    if pd.isnull( df['Bertelsmann Transformation index (BTI) - Ranking'].iloc[list(isocodes).index(country)]):
        df['Bertelsmann Transformation index (BTI) - Ranking'].iloc[list(isocodes).index(country)] = bti_data_keep[country]['rank']

In [285]:
# Save isocodes object for later use
with open('isocodes.pkl', 'wb') as handle:
  pickle.dump(isocodes, handle)

In [286]:
df.to_pickle("data.pkl")